In [2]:
!tritonserver --model-repository=/home/tiongkai/TiongKai/triton/model_repo/yoloseg-onnx/1/model.onnx --strict-model-config=false --exit-on-error=false --dump-model-config


/bin/bash: tritonserver: command not found


In [2]:
import onnx

model = onnx.load("/home/tiongkai/TiongKai/triton/model_repo/yoloseg-onnx/1/model.onnx")

print("Inputs:")
for i in model.graph.input:
    name = i.name
    dims = [d.dim_value for d in i.type.tensor_type.shape.dim]
    dtype = i.type.tensor_type.elem_type
    print(f"  - Name: {name}, Shape: {dims}, DType: {dtype}")

print("Outputs:")
for o in model.graph.output:
    name = o.name
    dims = [d.dim_value for d in o.type.tensor_type.shape.dim]
    dtype = o.type.tensor_type.elem_type
    print(f"  - Name: {name}, Shape: {dims}, DType: {dtype}")


Inputs:
  - Name: images, Shape: [0, 3, 0, 0], DType: 1
Outputs:
  - Name: output0, Shape: [0, 116, 0], DType: 1
  - Name: output1, Shape: [0, 32, 0, 0], DType: 1


In [18]:
!pip install opencv-python


  Using cached opencv_python-4.11.0.86-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
Using cached opencv_python-4.11.0.86-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (63.0 MB)


In [25]:
import numpy as np
import tritonclient.http as httpclient
import cv2

client = httpclient.InferenceServerClient(url="localhost:8000")
input_data = np.random.rand(1, 3, 512, 512).astype(np.float32)
input_data = cv2.imread('1.png').astype(np.float32)

resized = cv2.resize(input_data, (640, 640))  # (Width, Height)
chw = np.transpose(resized, (2, 0, 1))  # (3, 640, 640)

# Convert to float32 and normalize (if needed)
chw = chw.astype(np.float32) / 255.0  # Normalize to [0, 1] if model expects it

# Add batch dimension
input_tensor = np.expand_dims(chw, axis=0)  # (1, 3, 640, 640)

inputs = [
    httpclient.InferInput("images", input_tensor.shape, "FP32")
]
inputs[0].set_data_from_numpy(input_tensor)

outputs = [
    httpclient.InferRequestedOutput("output0"),
    httpclient.InferRequestedOutput("output1"),
]

response = client.infer("yoloseg-onnx", inputs=inputs, outputs=outputs)


In [26]:
result = response.as_numpy("output0")
print("Result:", result)

Result: [[[ 1.3986837e+01  2.9678478e+01  2.9032045e+01 ...  5.1945496e+02
    5.5032953e+02  5.9275366e+02]
  [ 5.0606556e+00  3.4676998e+00  3.6729958e+00 ...  5.8404578e+02
    5.9803540e+02  5.9740320e+02]
  [ 3.4946079e+01  6.0276146e+01  5.8369549e+01 ...  2.4009909e+02
    1.8918698e+02  1.3102319e+02]
  ...
  [-4.1507277e-01 -6.2845850e-01 -7.2458577e-01 ... -6.2909979e-01
   -5.8512980e-01 -6.4656460e-01]
  [ 2.3112774e-02  4.1996807e-02  7.7210724e-02 ... -1.7438345e-01
   -1.3832067e-01 -1.9488116e-01]
  [-7.4720830e-03 -2.4492612e-01 -2.1964540e-01 ...  3.2933634e-02
    3.8156107e-02  4.9060427e-02]]]


In [29]:
result[0][0].shape

(8400,)

# TensorRT

In [5]:
import tensorrt as trt

TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
with open("model_repo/yoloseg-engine/1/model.engine", "rb") as f, trt.Runtime(TRT_LOGGER) as runtime:
    engine = runtime.deserialize_cuda_engine(f.read())
    for i in range(engine.num_bindings):
        name = engine.get_binding_name(i)
        dtype = engine.get_binding_dtype(i)
        shape = engine.get_binding_shape(i)
        is_input = engine.binding_is_input(i)
        print("Input" if is_input else "Output", name, dtype, shape)


[07/04/2025-17:09:24] [TRT] [E] 1: [stdArchiveReader.cpp::StdArchiveReader::30] Error Code 1: Serialization (Serialization assertion magicTagRead == kMAGIC_TAG failed.Magic tag does not match)
[07/04/2025-17:09:24] [TRT] [E] 4: [runtime.cpp::deserializeCudaEngine::50] Error Code 4: Internal Error (Engine deserialization failed.)


AttributeError: 'NoneType' object has no attribute 'num_bindings'

In [1]:
from ultralytics import YOLO

model = YOLO('model_repo/yoloseg-engine/1/model.engine')

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.


AttributeError: 'YOLO' object has no attribute 'summary'